In [1]:
import numpy as np
import seaborn as sns

Based on write-up here: [link](https://mattferraro.dev/posts/poissons-equation)

In [13]:
size = 6

def start_state():
    mat = np.zeros((size, size))
    mat[:,-1]=1
    return mat

def update(mat):
    new_mat = np.zeros((size, size))
    for i in range(size):
        for j in range(size):
            if i == 0 or i == size-1 \
            or j == 0 or j == size-1:
                new_mat[i,j]=mat[i,j]
            else:
                new_mat[i,j]= 1/4 * (
                    mat[i-1,j] + mat[i+1,j] + mat[i,j-1] + mat[i,j+1]
                )
    return new_mat

def update_inplace(mat):
    for i in range(size):
        for j in range(size):
            if not (i == 0 or i == size-1 \
            or j == 0 or j == size-1):
                mat[i,j]= 1/4 * (
                    mat[i-1,j] + mat[i+1,j] + mat[i,j-1] + mat[i,j+1]
                )
    return mat

def make_pass(mat, func, rounds):
    for i in range(rounds):
        old_mat = mat.copy()
        mat = func(mat)
        if np.all(mat == old_mat):
            print(f"convergence reached at round {i}")
            break
    print(mat)

In [14]:
#declaring start states

matrix_1 = start_state()
matrix_2 = start_state()

In [15]:
make_pass(matrix_1, update, 1000)

convergence reached at round 173
[[0.         0.         0.         0.         0.         1.        ]
 [0.         0.04545455 0.10984848 0.22348485 0.45454545 1.        ]
 [0.         0.0719697  0.17045455 0.32954545 0.59469697 1.        ]
 [0.         0.0719697  0.17045455 0.32954545 0.59469697 1.        ]
 [0.         0.04545455 0.10984848 0.22348485 0.45454545 1.        ]
 [0.         0.         0.         0.         0.         1.        ]]


In [16]:
make_pass(matrix_2, update_inplace, 1000)

convergence reached at round 89
[[0.         0.         0.         0.         0.         1.        ]
 [0.         0.04545455 0.10984848 0.22348485 0.45454545 1.        ]
 [0.         0.0719697  0.17045455 0.32954545 0.59469697 1.        ]
 [0.         0.0719697  0.17045455 0.32954545 0.59469697 1.        ]
 [0.         0.04545455 0.10984848 0.22348485 0.45454545 1.        ]
 [0.         0.         0.         0.         0.         1.        ]]
